# <center>Welcome FSP solved with diverse heuristics</center>

## Data Utils

In [47]:
import numpy as np
import itertools
import time
import math
import pandas as pd
import matplotlib.pyplot as plt

In [48]:


'''
Reading the data from the benchmark file
'''

def read_flow_shop_data(file_path, machine_count, job_count):
    instances = []
    with open(file_path) as p:
        lines = p.readlines()
        line_count = len(lines)

        instance_count = line_count // (machine_count + 3)

        for i in range(instance_count):
            # recover the data of each instance
            params_line = lines[i * (machine_count + 3) + 1]
            job_count, machine_count, initial_seed, upper_bound, lower_bound = list(
                map(lambda x: int(x), params_line.split()))

            # processing_times = [list(map(int, lines[i * (machine_count + 3) + 3])) for line in lines]
            processing_times = np.array([list(map(lambda x: int(x), line.strip().split())) for
                                         line in lines[
                                                 i * (machine_count + 3) + 3:  # start
                                                 i * (machine_count + 3) + 3 + machine_count  # end
                                                 ]
                                         ])

            record = (machine_count, job_count, processing_times)
            instances.append(record)

    return instances



### Path Cost calculation function :
used to calculate the cost of current node, which is the correct cost starting for the actual path of executed jobs

In [49]:
def incremental_cost(machine_job_matrix, jobs_sequence):
    nb_jobs = len(jobs_sequence)
    nb_machines = machine_job_matrix.shape[0]

    incremental_cost = np.zeros((nb_machines, nb_jobs))

    # evaluate the first machines
    incremental_cost[0, 0] = machine_job_matrix[0][jobs_sequence[0]]

    for i in range(1, nb_jobs):
        incremental_cost[0][i] = incremental_cost[0][i - 1] + \
            machine_job_matrix[0][jobs_sequence[i]]

    # evaluate the rest of machines
    for i in range(1, nb_machines):
        incremental_cost[i, 0] = incremental_cost[i - 1, 0] + \
            machine_job_matrix[i, jobs_sequence[0]]
        for j in range(1, nb_jobs):
            incremental_cost[i, j] = machine_job_matrix[i, jobs_sequence[j]] + \
                max(incremental_cost[i - 1, j], incremental_cost[i, j - 1])
    return incremental_cost[-1,-1]

### Gantt graph generator

In [50]:
def generate_gantt_chart(current_instance, solution):
    plt.figure(figsize=(20, 12))
    df = pd.DataFrame(columns=['Machine', 'Job', 'Start', 'Finish'])

    machines, jobs = current_instance.shape
    machine_times = np.zeros((machines, jobs))
    start_time_m = np.zeros(machines)
    for job in solution:

        for machine_index in range(machines):
            start_time = start_time_m[machine_index]
            if machine_index > 0:
                start_time = max(start_time, start_time_m[machine_index-1])
            end_time = start_time + \
                current_instance[machine_index, job]
            start_time_m[machine_index] = end_time

            df = pd.concat([df, pd.DataFrame({'Machine': f'Machine {machine_index + 1}',
                                                'Job': f'Job {job + 1}',
                                                'Start': start_time,
                                                'Finish': end_time}, index=[0])], ignore_index=True)

            machine_times[machine_index, job] = end_time

    colors = plt.cm.tab10.colors
    for i, machine_index in enumerate(range(machines)):
        machine_df = df[df['Machine'] == f'Machine {machine_index + 1}']
        plt.broken_barh([(start, end - start) for start, end in zip(machine_df['Start'], machine_df['Finish'])],
                        (i * 10, 9), facecolors=[colors[j % 10] for j in range(jobs)], edgecolor='black')

    plt.xlabel('Time')
    plt.yticks([i * 10 + 4.5 for i in range(machines)],
                [f'Machine {i + 1}' for i in range(machines)])
    plt.show()

## Heuristics

### Johnson **n** jobs **2** machines

In [51]:
def johnson_method(processing_times):
    jobs, machines = processing_times.shape
    copy_processing_times = processing_times.copy()
    maximum = processing_times.max() + 1
    m1 = []
    m2 = []
    
    if machines != 2:
        raise Exception("Johson method only works with two machines")
        
    for i in range(jobs):
        minimum = copy_processing_times.min()
        position = np.where(copy_processing_times == minimum)
        
        if position[1][0] == 0:
            m1.append(position[0][0])
        else:
            m2.insert(0, position[0][0])
        
        copy_processing_times[position[0][0]] = maximum
        
    return m1+m2

### Test Johnson

In [52]:
# Generate a random example to work with 7 jobs and 2 machines
rnd_data = np.random.randint(size=(7,2), low=5, high=23)
print(rnd_data, "\n")

start_time = time.time()
sol = johnson_method(rnd_data)
elapsed_time = time.time() - start_time

print(f'Best sequence found by Johnson is {sol} with a makespan of {incremental_cost(np.array(rnd_data).T,sol)}')
print("Elapsed time:", elapsed_time, "seconds")

[[ 5 16]
 [ 5  9]
 [18 18]
 [ 5  5]
 [16 18]
 [ 9 19]
 [15 19]] 

Best sequence found by Johnson is [0, 1, 3, 5, 6, 4, 2] with a makespan of 109.0
Elapsed time: 0.0 seconds


### CDS Heuristic [Campbell,Dudek and Smith ] : **n** jobs **m** machines (1970)

In [180]:
# python code for CDS heuristic

def cds_heuristic(processing_times):
    
    
    nb_machines, nb_jobs = processing_times.shape


    best_cost = math.inf

    
    machine_1_times = np.zeros((nb_jobs,1))
    machine_2_times = np.zeros((nb_jobs,1))
    
    
    # iterate through the nb_machines-1 auxiliary n-job 2-machines problems



    for k in range(nb_machines -1):
        machine_1_times[:,0] += processing_times[:][k]
        machine_2_times[:,0] += processing_times[:][-k-1]
        
        jn_times = np.concatenate((machine_1_times, machine_2_times), axis=1)
        seq = johnson_method(jn_times)
        cost = incremental_cost(jn_times.T,seq)
        if cost < best_cost:
            best_cost = cost
            best_seq = seq
    
    return best_seq, best_cost


### Test CDS

In [199]:
rnd_data = np.random.randint(size=(8,5), low=10, high=50)
print(rnd_data, "\n")
cds_heuristic(rnd_data.T)
    


[[44 27 21 25 12]
 [32 36 25 35 49]
 [39 48 36 33 30]
 [41 15 40 32 43]
 [31 41 17 48 12]
 [38 14 35 43 21]
 [37 40 35 22 32]
 [42 26 31 24 17]] 



([1, 3, 6, 2, 5, 7, 4, 0], 316.0)

In [196]:
instances = read_flow_shop_data('../Lab1/data/tai20_5.txt', 5, 20)
comman_instance = instances[0][2]

In [198]:
start_time = time.time()
best_solution,best_cost  = cds_heuristic(comman_instance)


best_cost = incremental_cost(comman_instance,best_solution)

end_time = time.time()

elapsed_time = end_time - start_time

print(f'Results of CDS:')
print(f'Best sequence is {best_solution} with a makespan of {best_cost}.')
print(f'Elapsed time of {elapsed_time} seconds.')



Results of CDS:
Best sequence is [14, 2, 8, 13, 16, 7, 6, 0, 18, 3, 10, 15, 11, 1, 4, 5, 19, 17, 9, 12] with a makespan of 1390.0.
Elapsed time of 0.0 seconds.


### NEH Algorithm

In [160]:
def order_jobs_in_descending_order_of_total_completion_time(processing_times):
    total_completion_time = processing_times.sum(axis=1)
    return np.argsort(total_completion_time, axis=0).tolist()

In [159]:
def insertion(sequence, position, value):
    new_seq = sequence[:]
    new_seq.insert(position, value)
    return new_seq

In [167]:
def neh_algorithm(processing_times):
    ordered_sequence = order_jobs_in_descending_order_of_total_completion_time(processing_times)
    # Define the initial order
    J1, J2 = ordered_sequence[:2]
    sequence = [J1, J2] if incremental_cost(processing_times.T,[J1, J2]) < incremental_cost(processing_times.T,[J2, J1]) else [J2, J1]
    del ordered_sequence[:2]
    # Add remaining jobs
    for job in ordered_sequence:
        Cmax = float('inf')
        best_sequence = []
        for i in range(len(sequence)+1):
            new_sequence = insertion(sequence, i, job)
            Cmax_eval = incremental_cost(processing_times.T,new_sequence)
            if Cmax_eval < Cmax:
                Cmax = Cmax_eval
                best_sequence = new_sequence
        sequence = best_sequence
    return sequence, Cmax

In [169]:
start_time = time.time()
best_solution,best_cost  = neh_algorithm(instances[6][2].T)

end_time = time.time()

elapsed_time = end_time - start_time

print(f'Results of CDS:')
print(f'Best sequence is {best_solution} with a makespan of {best_cost}.')
print(f'Elapsed time of {elapsed_time} seconds.')


Results of CDS:
Best sequence is [4, 2, 19, 10, 7, 5, 3, 8, 1, 12, 6, 18, 16, 9, 14, 15, 0, 17, 13, 11] with a makespan of 1284.0.
Elapsed time of 0.0164792537689209 seconds.
